In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


  We will use adult census data to predict whether a person is making above $500,000 a year.
  Then, we use cross validation on our model to see which oneis performing better.

In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, KFold


In [22]:
df = pd.read_csv('/content/drive/MyDrive/projects/Cross-validation/adult.csv')

In [23]:
df.isnull().sum()

age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
income            0
dtype: int64

In [24]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [25]:
df.replace('?', np.nan, inplace=True)

In [26]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [9]:
df.isnull().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64

In [27]:
df.fillna(df.mean(), inplace = True)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

So in the above we are getting error coz we can't use mean for both numeric and non-numeric. for numeric we use mean, for categorical mode. so need to create seperately

In [28]:
#identitfy and seperate nueric and non-numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns
non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns

In [29]:
# Fill missing values in numeric columns with the mean
df[numeric_columns] = df[numeric_columns].apply(lambda col: col.fillna(col.mean()), axis=0)

In [30]:
# Fill missing values in non-numeric columns with the mode
for col in non_numeric_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [31]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Private,77053,HS-grad,9,Widowed,Prof-specialty,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Private,186061,Some-college,10,Widowed,Prof-specialty,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [32]:
#convert sex value to 0 and 1
df['sex'] = df['sex'].map({"Male": 0, "Female":1})

#create married column - yes(1) or no(0)
df['marital.status'] = df['marital.status'].replace(['Never-married','Divorced','Separated','Widowed'],0)
df['marital.status'] = df['marital.status'].replace(['Married-civ-spouse','Married-spouse-absent','Married-AF-spouse'],1)

df['marital.status'] = df['marital.status'].astype(int)

In [33]:
df.drop(labels = ["workclass", "education", "relationship", "race","native.country","occupation"], axis =1, inplace =True)

In [34]:
df.head()

,age,fnlwgt,education.num,marital.status,sex,capital.gain,capital.loss,hours.per.week,income
0,90,77053,9,0,1,0,4356,40,<=50K
1,82,132870,9,0,1,0,4356,18,<=50K
2,66,186061,10,0,1,0,4356,40,<=50K
3,54,140359,4,0,1,0,3900,40,<=50K
4,41,264663,10,0,1,0,3900,40,<=50K


In [35]:
x = df.drop('income', axis =1)
y = df['income']

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 0)

In [39]:
models = []
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))

In [40]:
models

[('LR', LogisticRegression()), ('KNN', KNeighborsClassifier())]

In [41]:
results= dict()
for name, model in models:
  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring='accuracy')
  results[name]= (cv_results.mean(), cv_results.std())
print()
print("name   results.mean    results.std")
for key, value in results.items():
  print(key, value)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



name   results.mean    results.std
LR (0.7969133858732026, 0.004770608928591671)
KNN (0.7725735543427947, 0.005140994125380305)


so mean and std is less in KNN so KNN would be better model to choose.